<center><h1>Machine Learning <br>Assignment 4</h1></center>

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

<h3>1.	We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefcients in two diferent ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis.  </h3>


In [ ]:
# 1. Load dataset
data = pd.read_csv("./Default.csv")
data['default'] = data['default'].map({'Yes': 1, 'No': 0})

<h4>(a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors. </h4>


<p>(a) Logistic regression with GLM</p>

In [ ]:

X = data[['income', 'balance']]
X = sm.add_constant(X)
y = data['default']

glm_model = sm.GLM(y, X, family=sm.families.Binomial()).fit()
print(glm_model.summary())
se_glm = glm_model.bse[['income','balance']]
print("\nStandard Errors (GLM):")
print(se_glm)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Tue, 23 Sep 2025   Deviance:                       1579.0
Time:                        10:25:04   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.0

<h3>b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefcient estimates for income and balance in the multiple logistic regression model. </h3>

In [8]:
def boot_fn(data, index):
    sample = data.iloc[index]
    X_boot = sm.add_constant(sample[['income','balance']])
    y_boot = sample['default']
    model_boot = sm.GLM(y_boot, X_boot, family=sm.families.Binomial()).fit()
    return model_boot.params[['income', 'balance']]

<h3>(c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefcients for income and balance. </h3>

In [9]:
np.random.seed(42)
B = 1000
boot_estimates = np.zeros((B,2))

for i in range(B):
    idx = resample(np.arange(len(data)), replace=True, n_samples=len(data))
    boot_estimates[i] = boot_fn(data, idx)

se_bootstrap = boot_estimates.std(axis=0)

print("\nStandard Errors (Bootstrap):")
print(pd.Series(se_bootstrap, index=['income','balance']))


Standard Errors (Bootstrap):
income     0.000005
balance    0.000232
dtype: float64


<h3>(d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.</h3>

In [10]:
print("\nComparison of SEs:")
print("GLM:", se_glm.values)
print("Bootstrap:", se_bootstrap)


Comparison of SEs:
GLM: [4.98524546e-06 2.27381384e-04]
Bootstrap: [4.96858684e-06 2.32092242e-04]


<h3>2.	We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis. </h3>

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

np.random.seed(42)

<h4>(a) Fit a logistic regression model that uses income and balance to predict default. </h4>

In [ ]:
X = data[['income', 'balance']]
y = data['default']

log_reg_full = LogisticRegression(max_iter=1000, random_state=42)
log_reg_full.fit(X, y)

print("Full model coefficients:")
print("Intercept:", log_reg_full.intercept_)
print("Income coef:", log_reg_full.coef_[0][0])
print("Balance coef:", log_reg_full.coef_[0][1])


Full model coefficients:
Intercept: [-11.54046792]
Income coef: 2.0808974142861165e-05
Balance coef: 0.0056471026485558845


<h4>(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:  <br>
i. Split the sample set into a training set and a validation set. </h4>


In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.5, random_state=42
    )

<h4>ii. Fit a multiple logistic regression model using only the training observations. </h4>

In [14]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


<h4>iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5. </h4>

In [15]:
y_prob = log_reg.predict_proba(X_valid)[:, 1]
y_pred = (y_prob > 0.5).astype(int)

<h4>iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified. </h4>

In [16]:
error = 1 - accuracy_score(y_valid, y_pred)
print(f"Validation Set Error: {error}")

Validation Set Error: 0.025800000000000045


<h4>(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.</h4>

In [ ]:
def validation_error(seed):
	X_train, X_valid, y_train, y_valid = train_test_split(
		X, y, test_size=0.5, random_state=seed
	)
	log_reg = LogisticRegression(max_iter=1000, random_state=seed)
	log_reg.fit(X_train, y_train)
	y_prob = log_reg.predict_proba(X_valid)[:, 1]
	y_pred = (y_prob > 0.5).astype(int)
	error = 1 - accuracy_score(y_valid, y_pred)
	return error

errors = [validation_error(1), validation_error(2), validation_error(3)]
print("\nValidation set errors:", errors)
print("Average validation error:", np.mean(errors))


Validation set errors: [0.025000000000000022, 0.024800000000000044, 0.024800000000000044]
Average validation error: 0.024866666666666704


<h2 style= "float:right">Kavinprasath.S.M   <br> 1P24CS011</h2>
